In [2]:
# Importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import pandas_profiling as pp

In [3]:
# Reading the data
book=pd.read_csv('Books.csv')
ratings=pd.read_csv('Ratings.csv').sample(30000)
users=pd.read_csv('Users.csv')

C:\Users\H\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Print the first 5 rows of book dataframe
book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
# printing the first 5 rows of ratings dataframe
ratings.head()

,User-ID,ISBN,Book-Rating
132511,30533,0375506039,0
952990,230522,0590418858,0
410113,98391,0756401275,10
949444,229899,0192833863,7
248480,56856,0373260962,0


In [6]:
# printing the first 5 rows of users dataframe
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
# printing the number of rows and columns for each dataframe
print('book dataframe has {} rows and {} columns'.format(book.shape[0],book.shape[1]))
print('users dataframe has {} rows and {} columns'.format(users.shape[0],users.shape[1]))
print('ratings dataframe has {} rows and {} columns'.format(ratings.shape[0],ratings.shape[1]))

book dataframe has 271360 rows and 8 columns
users dataframe has 278858 rows and 3 columns
ratings dataframe has 30000 rows and 3 columns


In [8]:
# merging the dataframes
user_ratings=ratings.merge(users,left_on='User-ID',right_on='User-ID')
book_user_ratings=user_ratings.merge(book,left_on='ISBN',right_on='ISBN')
book_user_ratings.shape

(26880, 12)

In [9]:
# creating our dataframe
data=book_user_ratings[['ISBN','User-ID','Book-Title','Book-Rating','Book-Author']]

In [10]:
# to avoid duplicating we will group the data by User-ID and Book-Title then calculate the mean ratings
data=data.groupby(['User-ID','Book-Title'])['Book-Rating'].mean().reset_index()
data

,User-ID,Book-Title,Book-Rating
0,9,Beloved (Plume Contemporary Fiction),6
1,17,Prophet,3
2,99,"Rich Dad, Poor Dad: What the Rich Teach Their ...",10
3,99,The Pillars of the Earth,3
4,160,O Deus Das Pequenas Coisas,0
...,...,...,...
26872,278681,A Judgement in Stone,7
26873,278771,Heartfire: The Tales of Alvin Marker V (Tales ...,0
26874,278851,Captain Underpants and the Wrath of the Wicked...,5
26875,278851,Covenant with the Vampire (Diaries of the Fami...,0


In [11]:
data=data[data['Book-Rating']!=0]

In [12]:
data

,User-ID,Book-Title,Book-Rating
0,9,Beloved (Plume Contemporary Fiction),6
1,17,Prophet,3
2,99,"Rich Dad, Poor Dad: What the Rich Teach Their ...",10
3,99,The Pillars of the Earth,3
5,228,My Antonia,9
...,...,...,...
26870,278663,A Woman's Place,5
26871,278668,LA Muerte Del Decano,5
26872,278681,A Judgement in Stone,7
26874,278851,Captain Underpants and the Wrath of the Wicked...,5


In [13]:
print('data dataframe has {} rows and {} columns'.format(data.shape[0],data.shape[1]))


data dataframe has 9988 rows and 3 columns


# Collaborative filtering


In [14]:
# Creating a pivot table
user_rating_pivot=data.pivot(index='User-ID',columns='Book-Title',values='Book-Rating')
user_rating_pivot.head()

Book-Title,"Goosebumps Monster Edition 1: Welcome to Dead House, Stay Out of the Basement, and Say Cheese and Die!",Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),"Summonings: Books of Magic, Vol. 2",!Arriba! Comunicacion y cultura,'N Sync,"--Olivetti, Moulinex, Chaffoteaux et Maury (Quaderns crema. Narrativa)","1, 2, 3","1,000 Places to See Before You Die",100 Best-Loved Poems (Dover Thrift Editions),100 Great Natural Remedies: Using Healing Plants at Home,...,"\15 to 1\"" Superchallenge""","\O\"" Is for Outlaw""","\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",e,flambeau@darkcorp.com,iI Paradiso Degli Orchi,one hundred years of solitude,stardust,why I'm like this : True Stories,Ã?Â?. Kolumnen.
User-ID,,,,,,,,,,,,,,,,,,,,,
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# calculating the sparsity
number_of_empty=user_rating_pivot.isnull().values.sum()
total_number=user_rating_pivot.size
sparsity=number_of_empty/total_number
sparsity

0.9998054813630288

In [16]:
# Normalize the data
# Get the average rating for each user 
aver_pivot=user_rating_pivot.mean(axis=1)
# Center each user's ratings around 0 and Fill in all missing values with 0s
user_rating_pivot=user_rating_pivot.sub(aver_pivot,axis=0).fillna(0)

In [17]:
from scipy.sparse.linalg import svds

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(user_rating_pivot)

In [18]:
# Convert sigma into a diagonal matrix
sigma = np.diag(sigma)

In [19]:
# Dot product of U and sigma
U_sigma = np.dot(U, sigma)
# Dot product of result and Vt
U_sigma_Vt = np.dot(U_sigma, Vt)

In [20]:
# Add back on the row means contained in avg_ratings
uncentered_ratings = U_sigma_Vt + aver_pivot.values.reshape(-1, 1)
# Create DataFrame of the results
calc_pred_ratings_df = pd.DataFrame(uncentered_ratings, 
                                    index=user_rating_pivot.index,
                                    columns=user_rating_pivot.columns
                                   )

In [21]:
# calculating the cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
# Generate the similarity matrix
similarities = cosine_similarity(calc_pred_ratings_df)
# Wrap the similarities in a DataFrame
user_base = pd.DataFrame(similarities, index=calc_pred_ratings_df.index, columns=calc_pred_ratings_df.index)

In [22]:
# Generate the similarity matrix
similarities1 = cosine_similarity(calc_pred_ratings_df.T)
# Wrap the similarities in a DataFrame
content_base = pd.DataFrame(similarities1, index=calc_pred_ratings_df.columns, columns=calc_pred_ratings_df.columns)

In [57]:
def similar_book(book_name,recommendation_num):
    cosine_similarity_series = content_base.loc[book_name]
    ordered_similarities = cosine_similarity_series.sort_values(ascending=False)
    ordered_similarities=pd.DataFrame(ordered_similarities)
    ordered_similarities=ordered_similarities.reset_index()
    return print(ordered_similarities['Book-Title'].head(recommendation_num))

In [65]:
similar_book('My Antonia',3)

0          The Hunt for Red October (Jack Ryan Novels)
1    Wish You Were Here (Mrs. Murphy Mysteries (Pap...
2    A Child's Book of Art: Great Pictures First Words
Name: Book-Title, dtype: object


In [87]:
def recommend(user_id):
    cosine_similarity_series = user_base.loc[user_id]
    ordered_similarities = cosine_similarity_series.sort_values(ascending=False)
    ordered_similarities=pd.DataFrame(ordered_similarities)
    ordered_similarities=ordered_similarities.reset_index()
    similar_user=ordered_similarities['User-ID'][0]
    recommendation=calc_pred_ratings_df.loc[similar_user].sort_values(ascending=False)
    recommendation=pd.DataFrame(recommendation)
    recommendation=recommendation.reset_index()
    return recommendation['Book-Title'].head()

In [93]:
recommend(17)

0     Goosebumps Monster Edition 1: Welcome to Dead...
1            Speak, Memory: An Autobiography Revisited
2                          Spilling Clarence : A Novel
3                                              Spiders
4    Spider Woman's Granddaughters: Traditional Tal...
Name: Book-Title, dtype: object